In [4]:
import pickle
import os
from numpy import random
import wfdb

import sys
sys.path.append('../libs')

import flacdb
import prepare_data

In [5]:
def get_downloaded_rec_segs():
    rec_segs = os.listdir(flacdb.ROOT)
    to_tuple = lambda i: flacdb.rec_seg_to_tuple(i.split('.')[0])
    S1 = {to_tuple(i) for i in rec_segs if '_x.flac' in i}
    S2 = {to_tuple(i) for i in rec_segs if '_x.hea' in i}
    S3 = {to_tuple(i) for i in rec_segs if '_y.hea' in i}
    S4 = {to_tuple(i) for i in rec_segs if '_y.flac' in i}
    S5 = {to_tuple(i) for i in rec_segs if '.hea' in i and '_x' not in i and '_y' not in i}
    S = S1 & S2 & S3 & S4 & S5
    return S

In [26]:
def get_available_rec_segs(): 
    S = []
    db = 'mimic3wdb'
    recs = wfdb.get_record_list(db)
    for rec in recs[:5]:
        S += wfdb.get_record_list(db + '/' + rec)
    S = {flacdb.rec_seg_to_tuple(i) for i in S if '_' in i}
    return S

In [7]:
recs = wfdb.get_record_list('mimic3wdb')

In [8]:
len(recs)

67830

In [13]:
%%time
S_downloaded = get_downloaded_rec_segs()

CPU times: user 7.96 s, sys: 2.82 s, total: 10.8 s
Wall time: 15.9 s


In [23]:
with open('/scr1/mimic/files_lists/available.json', 'wb') as f:
    pickle.dump(S_available, f)

In [25]:
len(pickle.load(open('/scr1/mimic/files_lists/available.json', 'rb')))

101

In [20]:
ls /scr1/mimic/files_lists/

part_0.txt  part_2.txt  part_4.txt  part_6.txt  part_8.txt
part_1.txt  part_3.txt  part_5.txt  part_7.txt  part_9.txt


In [10]:
%%time
S_available = get_available_rec_segs()

CPU times: user 76 ms, sys: 12 ms, total: 88 ms
Wall time: 5.03 s


In [42]:
S_ = wfdb.get_record_list('mimic3wdb')
S_ = {int(i.split('/')[1]) for i in S_}

In [44]:
all(i[0] in S_ for i in rec_segs)

True

In [3]:
hdrs = pickle.load(open('/scr1/mimic/headers_.pkl', 'rb'))
for i in hdrs:
    if i['sig_name'] is None:
        i['sig_name'] = []
bp_sigs = ['ABP', 'CVP', 'PAP', 'ICP']
hdrs_filtered = [
    i for i in hdrs
    if any(j in i['sig_name'] for j in bp_sigs)
    and 'n' not in i['record_name']    
]
S = {flacdb.rec_seg_to_tuple(i['record_name']) for i in hdrs_filtered}

In [5]:
hdrs_m = pickle.load(open('/scr1/mimic/headers_matched_.pkl', 'rb'))
S_m = {
    flacdb.rec_seg_to_tuple(i['record_name']) 
    for i in hdrs_m if i['record_name'][0] != 'p'
}

In [7]:
len(S_m - S)

531338

In [6]:
len(S - S_m)

401217

In [3]:
def get_rec_segs_with_bp():
    hdrs = pickle.load(open('/scr1/mimic/headers_.pkl', 'rb'))
    for i in hdrs:
        if i['sig_name'] is None:
            i['sig_name'] = []
    bp_sigs = ['ABP', 'CVP', 'PAP', 'ICP']
    hdrs_filtered = [
        i for i in hdrs
        if any(j in i['sig_name'] for j in bp_sigs)
        and 'n' not in i['record_name']    
    ]
    S = {flacdb.rec_seg_to_tuple(i['record_name']) for i in hdrs_filtered}
    return S

In [56]:
def get_matched_rec_segs():
    hdrs = pickle.load(open('/scr1/mimic/headers_matched_.pkl', 'rb'))
    S = {
        flacdb.rec_seg_to_tuple(i['record_name']) 
        for i in hdrs if i['record_name'][0] != 'p'
    }
    return S

In [138]:
S_matched = get_matched_rec_segs()
S_downloaded = get_downloaded_rec_segs()

In [58]:
len(S_matched - S_downloaded)

595479

In [82]:
rec_segs[5]

(3000003, 9)

In [141]:
def get_new_rec_segs():
    S = []
    for i in range(10):
        root = '/scr1/mimic/new_data_raw/3{}'.format(i)
        recs = os.listdir(root)
        for j in recs:
            segs = os.listdir(root + '/' + j)
            S += [(int(j), int(s.split('_')[1].split('.')[0])) for s in segs]
    return set(S)

In [134]:
def get_available_rec_segs():
    S = []
    for i in range(10):
        root = '/scr1/mimic/headers/3{}'.format(i)
        recs = os.listdir(root)
        for j in recs:
            if 'RECORDS' not in j:
                segs = os.listdir(root + '/' + j)
                segs = [s.split('_')[1][:-4] for s in segs if '_' in s]
                segs = [s for s in segs if len(s) == 4]
                S += [(int(j), int(s)) for s in segs]
    return set(S)

In [566]:
def get_matched_rec_segs():
    hdrs = glob.glob('/scr1/mimic/headers/matched/p0*/*/*.hea')
    hdrs = [i.split('/')[-1].split('.')[0] for i in hdrs]
    rec_segs = [
        i.split('_') for i in hdrs 
        if 'layout' not in i and 'p' not in i
    ]
    rec_segs = [(int(i), int(j)) for i, j in rec_segs]
    return sorted(set(rec_segs))

In [561]:
S_available = get_available_rec_segs()

In [562]:
len(S_available)

2269364

In [563]:
import glob

In [564]:
matched_headers = glob.glob('/scr1/mimic/headers/matched/p0*/*/*.hea')

In [565]:
matched_headers[0]

'/scr1/mimic/headers/matched/p06/p067384/3697767_layout.hea'

In [136]:
len(S_matched - S_available)

3934

In [133]:
len(S_matched_)

246870

In [126]:
len(S_available)

670840

In [101]:
S_new = get_new_downloaded_rec_segs()

In [102]:
len(S_new)

66261

In [144]:
len(S_matched & S_available - S_downloaded - S_new)

566247

In [95]:
S_d2[:5]

[(3000003, 1), (3000003, 2), (3000003, 3), (3000003, 4), (3000003, 5)]

In [85]:
files[0]

'3000484_0008.hea'

In [560]:
S_new = get_new_rec_segs()
S = (S_matched & S_available) - S_downloaded - S_new
rec_segs = sorted(S)
len(rec_segs)

591542

In [165]:
_to_file = lambda x: '{}/{}_{}'.format(x[0], x[0], str(x[1]).zfill(4))
_to_files = lambda x: [_to_file(x) + '.dat', _to_file(x) + '.hea']
filenames = [j for i in rec_segs for j in _to_files(i)]

In [62]:
import random

In [167]:
for part in range(10):
    with open('/scr1/mimic/files_lists/part_{}.txt'.format(part), 'w') as f:
        lines = [i for i in filenames if i[1] == str(part)]
        random.shuffle(lines)
        f.write('\n'.join(lines))

In [161]:
ls -lh /scr1/mimic/files_lists/

total 28M
-rw-r--r-- 1 kuprel users 2.8M Apr  6 22:21 part_0.txt
-rw-r--r-- 1 kuprel users 2.9M Apr  6 22:21 part_1.txt
-rw-r--r-- 1 kuprel users 2.5M Apr  6 22:21 part_2.txt
-rw-r--r-- 1 kuprel users 2.5M Apr  6 22:21 part_3.txt
-rw-r--r-- 1 kuprel users 2.8M Apr  6 22:21 part_4.txt
-rw-r--r-- 1 kuprel users 2.7M Apr  6 22:21 part_5.txt
-rw-r--r-- 1 kuprel users 3.3M Apr  6 22:21 part_6.txt
-rw-r--r-- 1 kuprel users 3.2M Apr  6 22:21 part_7.txt
-rw-r--r-- 1 kuprel users 2.0M Apr  6 22:21 part_8.txt
-rw-r--r-- 1 kuprel users 2.7M Apr  6 22:21 part_9.txt


In [532]:
files = []
for i in range(10):
    root = '/scr1/mimic/new_data_raw/3{}'.format(i)
    recs = os.listdir(root)
    for j in recs:
        files_new = os.listdir(root + '/' + j)
        files += [root + '/' + j + '/' + f for f in files_new]
        
nz_dat_count = sum(i[-4:] == '.dat' and os.path.getsize(i) > 0 for i in files)
dat_count = sum(i[-4:] == '.dat' for i in files)
flac_count = sum(i[-5:] == '.flac' for i in files)

print(dat_count, 'dat')
print(nz_dat_count, 'nonzero dat')
print(flac_count, 'flac')
print(nz_dat_count + flac_count - dat_count, 'excess')
# print(sum(i[-4:] in ['.dat', 'flac']  for i in files), 'total')

459767 dat
0 nonzero dat
632726 flac
172959 excess


In [549]:
import glob

In [555]:
for i in glob.glob('/scr1/mimic/new_data/*_y.*'):
    os.remove(i)

In [556]:
files = os.listdir('/scr1/mimic/new_data/')

In [546]:
sum('_x' in i and 'hea' in i for i in files)

3721

In [557]:
sum(i[-4:] == '.hea' for i in files)

632726

In [559]:
sum(i[-4:] == 'flac' for i in files)

632726

In [528]:
!df -h | grep scr1

/dev/md0                         1.8T  1.7T   73G  96% /scr1


In [224]:
flac_files = [i for i in files if '.flac' in i]

In [228]:
sum(os.path.isfile(i.replace('.flac', '.dat')) for i in flac_files)

111943

In [227]:
for i in flac_files:
    j = i.replace('.flac', '.dat')
    if not os.path.isfile(j):
        open(j, 'a').close()

In [230]:
os.path.isfile(files[0].replace('.flac', '.dat'))

True

In [234]:
os.path.getsize(files[0].replace('.flac', '.dat'))

0

In [346]:
!du -h -d 1 /scr1/mimic/headers

1.1G	/scr1/mimic/headers/33
976M	/scr1/mimic/headers/38
^C


In [220]:
files[0]

'/scr1/mimic/new_data_raw/30/3011280/3011280_0008.flac'

In [80]:
[sum(str(j[0])[1] == str(i) for j in rec_segs) for i in range(10)]

[57188, 58991, 57192, 56929, 58561, 56879, 69186, 70032, 47825, 62696]

In [181]:
!du -h -d 1 /scr1/mimic/new_data_raw

47G	/scr1/mimic/new_data_raw/33
62G	/scr1/mimic/new_data_raw/38
24G	/scr1/mimic/new_data_raw/32
727M	/scr1/mimic/new_data_raw/36
569M	/scr1/mimic/new_data_raw/34
67G	/scr1/mimic/new_data_raw/39
7.1G	/scr1/mimic/new_data_raw/35
707M	/scr1/mimic/new_data_raw/37
420M	/scr1/mimic/new_data_raw/31
1.6G	/scr1/mimic/new_data_raw/30
210G	/scr1/mimic/new_data_raw


In [53]:
!du -h -d 1 /scr1/mimic

888G	/scr1/mimic/waveforms
44G	/scr1/mimic/clinic
269G	/scr1/mimic/new_data_raw
8.4M	/scr1/mimic/files_lists
60M	/scr1/mimic/initial_data
1.2T	/scr1/mimic


In [45]:
len(files) / 2

66161.0

In [39]:
os.listdir('/scr1/mimic/new_data_raw/30/3072013')

[]

In [38]:
files[2]

'3072013'

In [30]:
os.listdir('/scr1/mimic/new_data_raw/')

['33', '38', '32', '36', '34', '39', '35', '37', '31', '30']